In [40]:
# Goal: determine which nodes count as "having boba"
import osmnx as ox
import geopandas as gpd
import pickle
import json
import pandas as pd

## Read data

This shouldn't take longer than ~40 sec.

In [9]:
with open("./data/seattle_networkx_graph.pkl", "rb") as fin:
    G = pickle.load(fin)

In [10]:
nodes, edges = ox.graph_to_gdfs(G)

In [11]:
nodes.head()

,y,x,highway,street_count,ref,geometry
osmid,,,,,,
29449863,47.643976,-122.304067,traffic_signals,4,NaN,POINT (-122.30407 47.64398)
29462909,47.641088,-122.297930,NaN,3,NaN,POINT (-122.29793 47.64109)
29464223,47.643918,-122.304207,traffic_signals,3,NaN,POINT (-122.30421 47.64392)
29464742,47.658325,-122.301425,NaN,3,NaN,POINT (-122.30143 47.65832)
29464746,47.656011,-122.302121,NaN,3,NaN,POINT (-122.30212 47.65601)


In [12]:
edges.head()

osmid  oneway   lanes  \
u        v           key                                                     
29449863 3391701883  0                           332060236   False       3   
         10791184626 0                           424636850   False       2   
         29464223    0                           426250827   False       2   
         2247309277  0    [1179848483, 4636259, 361443212]   False  [2, 4]   
29462909 3683076717  0                           364254479   False       2   

                             ref                            name    highway  \
u        v           key                                                      
29449863 3391701883  0    WA 513        East Montlake Place East  secondary   
         10791184626 0       NaN  East Lake Washington Boulevard  secondary   
         29464223    0       NaN                             NaN  secondary   
         2247309277  0    WA 513         Montlake Boulevard East    primary   
29462909 3683076717  0       NaN  Lake Washington Boulevard East  secondary   

                         maxspeed      junction reversed   length bridge  \
u        v           key                                                   
29449863 3391701883  0     25 mph  intersection     True    8.550    NaN   
         10791184626 0     25 mph           NaN     True    8.411    NaN   
         29464223    0        NaN  intersection    False   12.285    NaN   
         2247309277  0     30 mph           NaN    False  183.479    yes   
29462909 3683076717  0     25 mph           NaN    False   17.682    NaN   

                                                                   geometry  \
u        v           key                                                      
29449863 3391701883  0    LINESTRING (-122.30407 47.64398, -122.30400 47...   
         10791184626 0    LINESTRING (-122.30407 47.64398, -122.30397 47...   
         29464223    0    LINESTRING (-122.30407 47.64398, -122.30421 47...   
         2247309277  0    LINESTRING (-122.30407 47.64398, -122.30422 47...   
29462909 3683076717  0    LINESTRING (-122.29793 47.64109, -122.29795 47...   

                         access service width tunnel  
u        v           key                              
29449863 3391701883  0      NaN     NaN   NaN    NaN  
         10791184626 0      NaN     NaN   NaN    NaN  
         29464223    0      NaN     NaN   NaN    NaN  
         2247309277  0      NaN     NaN   NaN    NaN  
29462909 3683076717  0      NaN     NaN   NaN    NaN

In [62]:
# Hack from https://stackoverflow.com/questions/71907567/valueerror-geodataframe-does-not-support-multiple-columns-using-the-geometry-co
boba_gdf = gpd.read_file("./data/seattle_boba_shops.csv",
                         GEOM_POSSIBLE_NAMES="geometry", 
                         KEEP_GEOM_COLUMNS="NO")
boba_gdf = boba_gdf[["id", "location", "geometry"]]
boba_gdf.head()

,id,location,geometry
0,ozuMi1zz9jJkkikPLqNNSA,"{'address1': '600 5th Ave S', 'address2': 'Ste...",POINT (-122.32667 47.59680)
1,M4BjcVhOEjVgeF8EeSjZPQ,"{'address1': '519 6th Ave S', 'address2': '', ...",POINT (-122.32656 47.59765)
2,rFC0LUPyhgyP7UC9ehkrHw,"{'address1': '618 S Weller St', 'address2': ''...",POINT (-122.32529 47.59771)
3,I4pK6zZ4QwpmwVSTcDhH7A,"{'address1': '619 S King St', 'address2': None...",POINT (-122.32545 47.59823)
4,_L8WhDX0gwh1oMfpI7jdog,"{'address1': '679 S King St', 'address2': '', ...",POINT (-122.32394 47.59827)


In [68]:
for index, row in boba_gdf.iterrows():
    loc = eval(row[1])
    addr = f"{loc['address1']}, {loc['city']}, {loc['zip_code']}"

    print(f"Attempting to geocode {addr:>50}... ", end="")
    try:
        lat, long = ox.geocoder.geocode(addr)
        print(f"Found: {lat, long}")

        boba_gdf.iloc[index].x = long
        boba_gdf.iloc[index].y = lat

    except:
        print(f"Not found. Dropping row.")
        boba_gdf.drop(index=index, axis=0)

Attempting to geocode                      600 5th Ave S, Seattle, 98104... Found: (47.5972782, -122.327447)
Attempting to geocode                      519 6th Ave S, Seattle, 98104... Found: (47.5977153, -122.3265439)
Attempting to geocode                    618 S Weller St, Seattle, 98104... Found: (47.5976337, -122.3255237)
Attempting to geocode                      619 S King St, Seattle, 98104... Found: (47.5982488, -122.3254928)
Attempting to geocode                      679 S King St, Seattle, 98104... Found: (47.5982404, -122.3239502)
Attempting to geocode                  1043 S Jackson St, Seattle, 98104... Found: (47.5988089, -122.317458)
Attempting to geocode                        613 9th Ave, Seattle, 98104... Found: (47.6059309, -122.3253983)
Attempting to geocode                      1150 11th Ave, Seattle, 98122... Found: (47.6126016, -122.317964)
Attempting to geocode                      1416 10th Ave, Seattle, 98122... Found: (47.6133604, -122.3192083)
Attempting to

In [69]:
boba_gdf.head()

,id,location,geometry
0,ozuMi1zz9jJkkikPLqNNSA,"{'address1': '600 5th Ave S', 'address2': 'Ste...",POINT (-122.32667 47.59680)
1,M4BjcVhOEjVgeF8EeSjZPQ,"{'address1': '519 6th Ave S', 'address2': '', ...",POINT (-122.32656 47.59765)
2,rFC0LUPyhgyP7UC9ehkrHw,"{'address1': '618 S Weller St', 'address2': ''...",POINT (-122.32529 47.59771)
3,I4pK6zZ4QwpmwVSTcDhH7A,"{'address1': '619 S King St', 'address2': None...",POINT (-122.32545 47.59823)
4,_L8WhDX0gwh1oMfpI7jdog,"{'address1': '679 S King St', 'address2': '', ...",POINT (-122.32394 47.59827)
